In [10]:
import os 
import PIL
import cv2
import glob
import json
import time 
import pickle
import random

import numpy as np
import matplotlib.pyplot as plt

from fontTools.ttLib import TTFont

from PIL import Image,ImageDraw,ImageFont,ImageEnhance,ImageFilter

%matplotlib inline
    

In [11]:
output_dir_images = '/home/pengkang/Downloads/GenDataset/data/images/'
output_dir_mask = '/home/pengkang/Downloads/GenDataset/data/mask/'
output_dir_fg = '/home/pengkang/Downloads/GenDataset/data/fg/'
output_dir_bg = '/home/pengkang/Downloads/GenDataset/data/bg/'
pickle_path = '/home/pengkang/Downloads/GenDataset/data/'

font_dir = '/home/pengkang/Downloads/GenDataset/font/BaiZhouJingShuTi/*.ttf'
texture_path = glob.glob('/home/pengkang/Downloads/GenDataset/texture/*.jpg')
origin_images = glob.glob('/home/pengkang/Downloads/GenDataset/background/*')
font_paths = glob.glob(font_dir)

if not os.path.exists(output_dir_images):
    os.mkdir(output_dir_images)
if not os.path.exists(output_dir_mask):
    os.mkdir(output_dir_mask)
if not os.path.exists(output_dir_fg):
    os.mkdir(output_dir_fg)
if not os.path.exists(output_dir_bg):
    os.mkdir(output_dir_bg)

### Get Words

In [12]:
import words
simple_words = words.words

In [13]:
len(font_paths)

1

### set the range of font sizes

In [14]:
min_font_size = 24
max_font_size = 164

bound_min = 12 
bound_max = 24

## check wheher the chosen words in the library

In [15]:
def get_judge_word(font_path):
    font = TTFont(font_path)
    uni_list = font['cmap'].tables[0].ttFont.getGlyphOrder()
    uni_set = set(uni_list)
    def judge(s):
        ori = s
        a = s.encode("unicode-escape")[2:]
        s = 'uni' + ''.join([chr(i) for i in a]).upper()
        # print (ori,s in uni_set)
        return s in uni_set
    return judge

def judge_words(s,font_path):
    has_word = True
    f = get_judge_word(font_path)
    for i in range(len(s)):
        font = ImageFont.truetype(font=font_path,size=32)
        if np.array(font.getmask(s[i])).shape[0] == 0:
            has_word = False
    return (sum(map(f,s))==len(s) and has_word)

## add texture

In [16]:
def add_texture(mask_png,fg):
    now_path = random.choice(texture_path)
    texture = Image.open(now_path)
    texture_small  = texture.resize(mask_png.size[:2],resample=PIL.Image.BICUBIC)
    
    mask = np.array(mask_png,dtype=np.uint8)    
    fg_mask = 255 - np.zeros_like(mask,dtype=np.uint8)
    output_fg = cv2.seamlessClone(np.array(texture_small),np.array(fg),fg_mask,
                                  ((mask_png.size[0])//2,(mask_png.size[1])//2),cv2.NORMAL_CLONE)
    return Image.fromarray(output_fg)

## get boundary

In [17]:
def get_boundary(mask):
    mask = np.array(mask)
    mask[mask>0] = 1
    
    X = mask.sum(axis=0)
    Y = mask.sum(axis=1)
    
    X2 = X.cumsum().argmax()
    Y2 = Y.cumsum().argmax()
    X[X>0] = 1
    Y[Y>0] = 1
    X1 = X.argmax()
    Y1 = Y.argmax()
    return X1, Y1, X2, Y2

In [18]:
def get_mask(string,font,pos,img_size):
    mask_ori, offset = font.getmask2(string,'L')
    blank_image = np.zeros(img_size,dtype=np.uint8)
    mask_np = np.array(mask_ori).reshape(-1, font.size).astype(np.uint8)
    blank_image[pos[1]:pos[1] + mask_np.shape[0],\
                pos[0]:pos[0] + mask_np.shape[1]] = mask_np
    return blank_image, mask_np.shape

def put_word(string,draw,font,pos,img_size,color):
    mask_ori, offset = font.getmask2(string,'L')
    
    ########## offset calibration ########
    x = pos[0] - offset[0]
    y = pos[1] - offset[1]
    draw.text((x,y),string,font = font,fill = color)


def judge_over(mask1,mask2):
    return np.sum(mask2[mask1!=0].astype(int))==0

def get_pos(center,size):
    x=random.gauss(center[0],size)
    y=random.gauss(center[1],size)
    return [int(x), int(y)]

def get_new_pos(center,size):
    o_x = random.randint(-size[0]//2,size[0]//2)
    o_y = random.randint(-size[1]//2,size[1]//2)
    
    x = center[0] + o_x + (-(size[0]) if o_x < 0 else size[0]>>1)
    y = center[1] + o_y + (-(size[1]) if o_y < 0 else size[1]>>1)

    return [x, y]

def get_bigger(box):
    x1 = box[0] - random.randint(bound_min,bound_max)
    y1 = box[1] - random.randint(bound_min,bound_max)
    x2 = box[2] + random.randint(bound_min,bound_max)
    y2 = box[3] + random.randint(bound_min,bound_max)
    return (x1, y1, x2, y2)

def put_words(string,img,font_path,draw):
    boxes = []
    script_text = []
    font_name = os.path.basename(font_path)
    font_size = random.randint(min_font_size, max_font_size) 
    font = ImageFont.truetype(font=font_path,size=font_size)    
    color = tuple(np.random.randint(0,256,3))
    ####### pos #########
    w, h = img.size 
    pos = get_pos((w>>1,h>>1),font_size)
    
    ###################
    fg_image = (np.zeros_like(img)).astype(np.uint8)
    fg_image[:,:] = color
    fg_image = Image.fromarray(fg_image)
    ###################
    mask, (font_h,font_w) = get_mask(string[0], font, pos, (h,w))
    put_word(string[0], draw, font,pos, (h,w), color)
    script_text.append(string)

    ####### add more words #########
    pos_center = (pos[0] + font_w//2, pos[1]+font_h//2)
    for word in string[1:]:
        for pat in range(10):
            new_pos = get_new_pos(pos_center, (font_w,font_h))
            new_mask, _ = get_mask(word, font, new_pos, (h,w))
            if (judge_over(new_mask,mask)):
                put_word(word,draw,font,new_pos,(h,w),color)
                mask=mask|new_mask
                break
    
    #########################
    boxes.append(get_boundary(mask))
    bbox = get_bigger(boxes[0])
    bbox = crop_in_image(img.size,bbox)
    string_box = {"font_name":font_name,
                  "bboxes":bbox,
                  "string":string,
                  "font_size":font_size,
                  "bound_box":boxes,
                  "script_text":script_text}
    
    return img, string_box, mask, fg_image

def crop_in_image(img_size,bbox):
    w,h = img_size
    x1,y1,x2,y2 = bbox
    x1 = x1 if x1 > 0 else 0
    x2 = x2 if x2 < w-1 else w-1
    y1 = y1 if y1 > 0 else 0
    y2 = y2 if y2 < h-1 else h-1
    return (x1,y1,x2,y2)


def gen_text_randomSize(item,font_paths = font_paths, output_dir_images = output_dir_images,
                        output_dir_mask = output_dir_mask,output_dir_fg = output_dir_fg,
                        output_dir_bg = output_dir_bg,num_string=25):

    index,img_path = item
    img = cv2.imread(img_path)[...,::-1]

    label_dict = []
    coor_l = []
    cut_id = 0
    for i in range(num_string):
        font_path = random.choice(font_paths)
        Img = Image.fromarray(img)
        draw = ImageDraw.Draw(Img)
        fontname = os.path.basename(font_path)
        
        Flag = 10
        while Flag>0:
            string = ''.join(random.sample(simple_words,random.randint(1,5)))
            Flag -= 1 
            try:
                if (judge_words(string,font_path)):
                    Flag = -1
                    break
            except:
                print(font_path)
        if Flag != -1:
            continue
            
        try:
            Img,string_box,mask,fg = put_words(string, Img, font_path, draw)
        except Exception as e:
            print ('wrong 1',e)
            continue
            
        if Img is None:
            continue
        
        img_save_name = '%d_%d' %(index, cut_id) + '.png'
        
        bbbbox = string_box['bboxes']
        
        fg_tp = fg.crop(bbbbox)
       
        mask = mask[bbbbox[1]:bbbbox[3],bbbbox[0]:bbbbox[2]]
        mask_png = Image.fromarray(mask.astype(np.uint8))
        
        if random.random() < 0.8:
            fg_tp = add_texture(mask_png, fg_tp) 
            
        bg = Image.fromarray(img).crop(bbbbox)
        
        ############### Dataset Gen #############
        mask_np=mask/255.
        fg_np = np.array(fg_tp,dtype=np.float)
        bg_np = np.array(bg,dtype=np.float)
        new = fg_np*mask_np[...,np.newaxis]+(1.0-mask_np[...,np.newaxis])*bg_np+0.5
        new = new.astype(np.uint8)
        
        img_tp = Image.fromarray(new)
        ############### Dataset Gen #############
        
        img_tp.save(output_dir_images+img_save_name)
        mask_png.save(output_dir_mask+img_save_name)
        bg.save(output_dir_bg+img_save_name)
        fg_tp.save(output_dir_fg+img_save_name)
        total_bbox = []
        for i in range(len(string_box['bound_box'])):
            sub_total_bbox = []
            for j in range(4):
                sub_total_bbox.append(string_box['bound_box'][i][j] - string_box['bboxes'][j%2])
            total_bbox.append(sub_total_bbox)
        label_dict.append({'name':img_save_name,'box':total_bbox,'text':string_box['script_text']})
        cut_id+=1
        
        ###############show###############
        # uncomment this, when you want to return and plot
        # return img_tp,mask_png, bg ,fg_tp
        ###############show###############
        
    return label_dict



## Data Generation

In [19]:
from multiprocessing import Pool,cpu_count
os.system('rm '+ output_dir_bg + '*.png')
os.system('rm '+ output_dir_fg + '*.png')
os.system('rm '+ output_dir_images + '*.png')
os.system('rm '+ output_dir_mask + '*.png')

# origin_images = random.sample(origin_images,200)
indexes_images = [(i,path) for i,path in enumerate(origin_images)]
pool = Pool(5)
start = time.time()
%time result = pool.map(gen_text_randomSize,indexes_images)
end = time.time()
print(end-start,5)
label_dict = []
[label_dict.extend(item) for item in result]
print('done')

with open(pickle_path + '/new_image_list.pkl','wb') as f:
    pickle.dump(label_dict,f)
print (len(label_dict))

wrong 1 could not broadcast input array from shape (122,129) into shape (0,129)
CPU times: user 1.18 ms, sys: 4.15 ms, total: 5.32 ms
Wall time: 5.82 s
5.820515394210815 5
done
17
